In [9]:
import pandas as pd
import numpy as np
from pprint import pprint
import re
import os

from .src my_functions import *

SyntaxError: invalid syntax (<ipython-input-9-c5d4ff655576>, line 7)

In [ ]:
original_dataset=pd.read_csv('input/attacks.csv', encoding='latin-1')

At first sight we can see a 23 columns dataset. Some of them are dates, some of them are categorical data and links to files than can be useful if important information is missed.

In [ ]:
original_dataset.head()

We create a copy of the original dataset for security

In [ ]:
cleaned_dataset=original_dataset.copy()

# Looking for NaN values

First of all we are going to check the two last columns that appears to be only NaN values

Since the original data set has 25723 rows and the columns Unnamed:22 and Unnamed:23 have 25722  NaN values we can safely drop them.

In [ ]:
sum(cleaned_dataset['Unnamed: 22'].isna())

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns=["Unnamed: 23","Unnamed: 22"], axis=1)

In the cell bellow we can see that we are not almost near to detect all the missing values. Comparing the number of NaN to the number of data, it seems that there are lots of rows that are empty. Let's check that by creating a new column that shows how many NaN values has each row,

In [ ]:
cleaned_dataset.isna().sum()

In [ ]:
cleaned_dataset["Is_all_NaN"]=cleaned_dataset.isna().sum(axis=1)

cleaned_dataset["Is_all_NaN"].value_counts()

So we can see that there are a great number of columns that have no information or almost no information (columns with 22 or 21). We proceed to remove them. Afther, we will drop the generated "Is_all_NaN"

In [ ]:
cleaned_dataset=cleaned_dataset.dropna(axis=0, how='any', thresh=20, subset=None, inplace=False)

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns="Is_all_NaN")

There are still some NaN values, but we will with that later

In [ ]:
cleaned_dataset.isna().sum()

# Duplicated values

Now we are looking for rows that have duplicated values

In [ ]:
sum(cleaned_dataset.duplicated())

So there are not rows with duplicated values. Now we look for columns with duplicated values.

In [ ]:
cleaned_dataset[['Case Number','Date','Case Number.1','Case Number.2']]

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns=['Case Number.1','Case Number.2'], axis=1)

So with "Case Number" it was pretty easy to compare by sight, but it is more difficult to compare URLs by sight, so we will do it by checking with boolean logic.

In [ ]:
(cleaned_dataset['href formula']==cleaned_dataset['href']).value_counts()

It seems that both columns are the same but some mistakes had been made. Nonetheless, we cannot drop a column arbitrarily, so lets check the values that are not equal.

In [ ]:
pprint(cleaned_dataset[['href','href formula']].loc[cleaned_dataset['href']!=cleaned_dataset['href formula']].head())

When comparing the columns that have different values, we find that none of the URLs are operative anymore. The same is true for the columns that have equal values. This column can be dropped without losing any practical information.

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns=['href','href formula'])

'pdf' is a column listing reports files to which we do not hace access, so there is not usefull information for us there. On the other hand 'original order' is a column of integer numbers (formated as floats) that it is just de order in which the data were stored.

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns=['pdf','original order'])

Now we search for duplicated rows

In [ ]:
cleaned_dataset=cleaned_dataset.drop_duplicates()

Now we are finished (at least by the moment) with deleting rows and columns we are going to reset the dataframe indexes in sake of clarity

In [ ]:
cleaned_dataset=cleaned_dataset.reset_index(drop=True)

# Consistency of the data 

So now we are going to check whether each column has usefull and well formated information or no. This only can be done in a column by column basis, since no general criteria can be applied. Also, we will check if the datatype is the appropiate for this kind of data.

In [ ]:
cleaned_dataset.columns

In [ ]:
cleaned_dataset['Date'].value_counts()

Date column doesn't seem to have well formatted data. There is some uncompleted information about dates, but it still can be usefull in the future. Let's check next columns.

Year column is a more simple column and looks better than date column. However, when we check the minimun of the dataset, we have 0 values. Since we can hardly believe that we have data from year 0, let's try to complete this column with information from "Date" column

In [ ]:
cleaned_dataset['Year'].value_counts()

In [ ]:
cleaned_dataset['Date'].loc[cleaned_dataset['Year']==0]

We see that there is some related information in the column Date when Year is 0, but something comes to the eye. All the data that has 0 in year are acumulated at the end of the dataset(You can see this by looking to the indexes). Seems that those data are added at posteriori and has no consistency with the dataset. We will make further comprobations

There are 5253 values with a value different from 0 grouped in 200 categories (which is a very reasonable number for a period of time) but there are 81 values with a value equal to 0 from a total sample size of 5334

It's clear that those data are not realiable and we can drop it.

In [ ]:
cleaned_dataset=cleaned_dataset[cleaned_dataset.Year!=0]

Also Year data need to be casted to integer values

In [ ]:
cleaned_dataset['Year']=cleaned_dataset['Year'].astype(int)

In the column 'Type' we can see that values are razonably well categorized except little exceptions that can be easily fixed

In [ ]:
cleaned_dataset['Type'].value_counts()

In [ ]:
cleaned_dataset['Type']=cleaned_dataset['Type'].replace('Boat','Boating')
cleaned_dataset['Type']=cleaned_dataset['Type'].replace('Boatomg','Boating')

In [ ]:
cleaned_dataset['Type'].value_counts()

'Name' column doesn't seem to have many usefull information. Its five more frequent value are not names, but gender or age, qhich are stored in other columns. Also, particular names are not important for statistical questions we are trying to answer.

In [ ]:
cleaned_dataset['Name'].value_counts()

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns='Name')

Sex column is pretty well categorized but some data can be rescued

In [ ]:
cleaned_dataset['Sex '].value_counts()

In [ ]:
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('N','F')
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('M ','M')

In [ ]:
cleaned_dataset.loc[cleaned_dataset['Sex ']=='lli']

In [ ]:
cleaned_dataset.loc[cleaned_dataset['Sex ']=='.']

In [ ]:
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('lli','M')
cleaned_dataset=cleaned_dataset[cleaned_dataset['Sex ']!='.']
cleaned_dataset=cleaned_dataset.rename(columns={'Sex ':'Sex'})

'Age' column have some data values that can be fixed and a quite important cuantity of NaN wrong formatted.

In [ ]:
for i in cleaned_dataset.index:
    x=cleaned_dataset.loc[i,'Age']
    list_matches=re.findall(r'\d{1,2}',str(x))
    if len(list_matches)>=1:
        cleaned_dataset.loc[i,'Age']=int(sum(map(int,list_matches))/len(list_matches))
    else:
        cleaned_dataset.loc[i,'Age']=np.nan

'Injury' column has 3262 different values with no coding or standard, so we cannot extract statistical information from here. Anyhow, it will remain because it does have information.

In [ ]:
cleaned_dataset['Injury'].value_counts()

'Fatal (Y/N)' column is mostly well codificated but need some fixes

In [ ]:
cleaned_dataset['Fatal (Y/N)'].value_counts()

In [ ]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace(' N','N')

In [ ]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace('M','N')

In [ ]:
cleaned_dataset.loc[cleaned_dataset['Fatal (Y/N)']=='2017']

In [ ]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace('2017','N')
cleaned_dataset=cleaned_dataset.rename(columns={'Fatal (Y/N)':'Fatal'})

'Time' values are not standarized, so we will try to 

In [ ]:
cleaned_dataset['Time'].value_counts()

In [ ]:
cleaned_dataset.Time=cleaned_dataset.Time.apply(assign_time)

'Species' column is not pretended to be used in further analysis, so we will drop it. 

In [ ]:
cleaned_dataset=cleaned_dataset.drop(columns=['Species '])

'investigator or source' column has no use for statistical purposes but still has information that does have ofr individual cases

In [ ]:
cleaned_dataset=cleaned_dataset.reset_index(drop=True)

This, our dataset is ready to be exported.

In [ ]:
cleaned_dataset.to_csv('output/Sharks_attacks_cleaned.csv',index=False)